# 📊 Week 8: Sales Dashboard - Complete Demo
**Clogenai Academy – Data Pipeline Engineering Track**

## What This Notebook Does
1. ✅ Generates sample e-commerce data
2. ✅ Creates Gold-layer tables (dimension + fact)
3. ✅ Provides all 6 dashboard queries ready for visualization
4. ✅ Includes instructions for building the dashboard

**Total Time**: 15-20 minutes

---


## 🔧 Step 1: Setup Catalog and Schemas


In [ ]:
%sql
-- Use academy catalog
USE CATALOG academy;

-- Create schemas
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;


## 📦 Step 2: Generate Sample Data

Creating realistic e-commerce data:
- **12 products** across 3 categories (Electronics, Clothing, Home & Garden)
- **100 orders** over 3 months (Jan-Mar 2024)


In [ ]:
%sql
-- Create sample products
CREATE OR REPLACE TABLE academy.silver.products AS
SELECT 
    CONCAT('PROD', LPAD(CAST(id AS STRING), 3, '0')) AS product_id,
    CASE 
        WHEN id <= 3 THEN CONCAT('Laptop Model ', id)
        WHEN id <= 6 THEN CONCAT('Smartphone ', id - 3)
        WHEN id <= 9 THEN CONCAT('T-Shirt Style ', id - 6)
        ELSE CONCAT('Office Chair ', id - 9)
    END AS product_name,
    CASE 
        WHEN id <= 6 THEN 'Electronics'
        WHEN id <= 9 THEN 'Clothing'
        ELSE 'Home & Garden'
    END AS category,
    CASE 
        WHEN id <= 3 THEN 'Laptops'
        WHEN id <= 6 THEN 'Phones'
        WHEN id <= 9 THEN 'Apparel'
        ELSE 'Furniture'
    END AS subcategory,
    CASE 
        WHEN id <= 3 THEN 'TechBrand'
        WHEN id <= 6 THEN 'PhoneCo'
        WHEN id <= 9 THEN 'FashionHub'
        ELSE 'HomeComfort'
    END AS brand,
    CASE 
        WHEN id <= 3 THEN 800 + (id * 200)
        WHEN id <= 6 THEN 400 + ((id - 3) * 150)
        WHEN id <= 9 THEN 25 + ((id - 6) * 10)
        ELSE 150 + ((id - 9) * 50)
    END AS retail_price,
    CASE 
        WHEN id <= 3 THEN 600 + (id * 150)
        WHEN id <= 6 THEN 300 + ((id - 3) * 100)
        WHEN id <= 9 THEN 15 + ((id - 6) * 5)
        ELSE 100 + ((id - 9) * 30)
    END AS cost_price
FROM RANGE(1, 13);


In [ ]:
%sql
-- Verify products
SELECT * FROM academy.silver.products ORDER BY category, product_name;


In [ ]:
%sql
-- Create sample orders
CREATE OR REPLACE TABLE academy.silver.orders AS
SELECT 
    CONCAT('ORD', LPAD(CAST(id AS STRING), 4, '0')) AS order_id,
    CONCAT('CUST', LPAD(CAST(MOD(id * 7, 50) + 1 AS STRING), 3, '0')) AS customer_id,
    CONCAT('PROD', LPAD(CAST(MOD(id * 13, 12) + 1 AS STRING), 3, '0')) AS product_id,
    DATE_ADD('2024-01-01', CAST(MOD(id * 3, 90) AS INT)) AS order_date,
    CAST(MOD(id * 5, 3) AS INT) + 1 AS quantity,
    CASE 
        WHEN MOD(id, 10) > 8 THEN 'Cancelled'
        WHEN MOD(id, 20) = 19 THEN 'Returned'
        ELSE 'Completed'
    END AS order_status
FROM RANGE(1, 101);


In [ ]:
%sql
-- Verify orders
SELECT * FROM academy.silver.orders ORDER BY order_date LIMIT 10;


## 🌟 Step 3: Create Gold-Layer Tables


In [ ]:
%sql
-- Create product dimension
CREATE OR REPLACE TABLE academy.gold.dim_product AS
SELECT 
    product_id,
    product_name,
    category,
    subcategory,
    brand,
    retail_price,
    cost_price,
    retail_price - cost_price AS profit_margin
FROM academy.silver.products;


In [ ]:
%sql
-- Verify dimension
SELECT * FROM academy.gold.dim_product ORDER BY category, product_name;


In [ ]:
%sql
-- Create sales fact table
CREATE OR REPLACE TABLE academy.gold.fact_sales AS
SELECT 
    o.order_id,
    o.customer_id,
    o.product_id,
    o.order_date,
    o.quantity,
    p.retail_price AS unit_price,
    o.quantity * p.retail_price AS line_total,
    (o.quantity * p.retail_price) - (o.quantity * p.cost_price) AS profit
FROM academy.silver.orders o
JOIN academy.silver.products p ON o.product_id = p.product_id
WHERE o.order_status = 'Completed';


In [ ]:
%sql
-- Verify fact table
SELECT 
    COUNT(*) AS total_orders,
    SUM(line_total) AS total_revenue,
    SUM(profit) AS total_profit
FROM academy.gold.fact_sales;


---
# 📊 DASHBOARD QUERIES

The following queries are ready to be used for dashboard visualizations.

**Instructions**:
1. Run each query below to verify results
2. In Databricks SQL, create a new query for each
3. Copy the SQL from below
4. Add visualization to each query
5. Add all visualizations to a dashboard

---


## 📈 Query 1: Total Revenue (KPI Counter)

**Visualization Type**: Counter  
**Format**: Currency ($#,##0)  
**Color**: Green


In [ ]:
%sql
-- DASHBOARD QUERY 1: Total Revenue
SELECT 
    SUM(line_total) AS total_revenue
FROM academy.gold.fact_sales
WHERE order_date >= '2024-01-01'
  AND order_date <= '2024-03-31';


## 📦 Query 2: Total Orders (KPI Counter)

**Visualization Type**: Counter  
**Format**: Number (#,##0)  
**Color**: Blue


In [ ]:
%sql
-- DASHBOARD QUERY 2: Total Orders
SELECT 
    COUNT(DISTINCT order_id) AS total_orders
FROM academy.gold.fact_sales
WHERE order_date >= '2024-01-01'
  AND order_date <= '2024-03-31';


## 💵 Query 3: Average Order Value (KPI Counter)

**Visualization Type**: Counter  
**Format**: Currency ($#,##0.00)  
**Color**: Purple


In [ ]:
%sql
-- DASHBOARD QUERY 3: Average Order Value
SELECT 
    ROUND(AVG(line_total), 2) AS avg_order_value
FROM academy.gold.fact_sales
WHERE order_date >= '2024-01-01'
  AND order_date <= '2024-03-31';


## 📈 Query 4: Monthly Revenue Trend (Line Chart)

**Visualization Type**: Line Chart  
**X-Axis**: month (format as MMM YYYY)  
**Y-Axis**: revenue (format as currency)  
**Line Color**: Blue


In [ ]:
%sql
-- DASHBOARD QUERY 4: Monthly Revenue Trend
SELECT 
    DATE_TRUNC('month', order_date) AS month,
    SUM(line_total) AS revenue,
    COUNT(DISTINCT order_id) AS order_count
FROM academy.gold.fact_sales
WHERE order_date >= '2024-01-01'
  AND order_date <= '2024-03-31'
GROUP BY DATE_TRUNC('month', order_date)
ORDER BY month;


## 📊 Query 5: Revenue by Category (Bar Chart)

**Visualization Type**: Bar Chart (Horizontal)  
**X-Axis**: total_revenue (format as currency)  
**Y-Axis**: category  
**Colors**: Electronics (Blue), Clothing (Orange), Home & Garden (Green)


In [ ]:
%sql
-- DASHBOARD QUERY 5: Revenue by Category
SELECT 
    p.category,
    SUM(f.line_total) AS total_revenue,
    COUNT(DISTINCT f.order_id) AS order_count
FROM academy.gold.fact_sales f
JOIN academy.gold.dim_product p ON f.product_id = p.product_id
WHERE f.order_date >= '2024-01-01'
  AND f.order_date <= '2024-03-31'
GROUP BY p.category
ORDER BY total_revenue DESC;


## 🏆 Query 6: Top 10 Products (Table)

**Visualization Type**: Table  
**Columns**: product, category, units_sold, revenue, profit_margin_pct  
**Format**: Revenue as currency, profit_margin_pct as percentage


In [ ]:
%sql
-- DASHBOARD QUERY 6: Top 10 Products
SELECT 
    p.product_name AS product,
    p.category,
    SUM(f.quantity) AS units_sold,
    SUM(f.line_total) AS revenue,
    ROUND((SUM(f.profit) / SUM(f.line_total)) * 100, 1) AS profit_margin_pct
FROM academy.gold.fact_sales f
JOIN academy.gold.dim_product p ON f.product_id = p.product_id
WHERE f.order_date >= '2024-01-01'
  AND f.order_date <= '2024-03-31'
GROUP BY p.product_name, p.category
ORDER BY revenue DESC
LIMIT 10;


---
# 🎨 Building the Dashboard

## Step-by-Step Instructions

### 1. Navigate to Databricks SQL
- Click on **SQL** in the left sidebar
- Ensure your SQL warehouse is running

### 2. Create Queries
For each of the 6 queries above:
1. Click **Queries** → **Create Query**
2. Name the query (e.g., "Dashboard - Total Revenue")
3. Copy the SQL from the corresponding cell above
4. Paste and run to verify results
5. Click **Save**

### 3. Add Visualizations
For each saved query:
1. Click **Add Visualization**
2. Select the chart type (Counter, Line Chart, Bar Chart, or Table)
3. Configure settings:
   - **Query 1-3 (Counters)**: Select value column, set format, choose color
   - **Query 4 (Line Chart)**: X-axis = month, Y-axis = revenue
   - **Query 5 (Bar Chart)**: X-axis = total_revenue, Y-axis = category
   - **Query 6 (Table)**: All columns, format revenue and percentage
4. Click **Save**

### 4. Create Dashboard
1. Go to **Dashboards** → **Create Dashboard**
2. Name: "Sales Performance Dashboard"
3. Description: "Executive sales metrics for Q1 2024"
4. Click **Create**

### 5. Add Widgets to Dashboard
1. Click **Add** → **Visualization**
2. Select each visualization you created
3. Arrange in this layout:
   ```
   Row 1: [Total Revenue] [Total Orders] [Avg Order Value]
   Row 2: [Monthly Revenue Trend - full width]
   Row 3: [Revenue by Category] [Top 10 Products]
   ```
4. Resize widgets to match the layout

### 6. Polish & Share
1. Click **Done Editing**
2. Review the final dashboard
3. Click **Share** to grant access to team members
4. Set up auto-refresh schedule (optional)

---


## ✅ Verification Checklist

Before building the dashboard, verify:

- [ ] `academy.silver.products` table has 12 rows
- [ ] `academy.silver.orders` table has ~100 rows
- [ ] `academy.gold.dim_product` table has 12 rows
- [ ] `academy.gold.fact_sales` table has 85-95 rows (completed orders only)
- [ ] All 6 dashboard queries return results
- [ ] SQL warehouse is running

**If any checks fail, re-run the corresponding cells above.**


In [ ]:
%sql
-- Quick verification query
SELECT 
    'academy.silver.products' AS table_name,
    COUNT(*) AS row_count
FROM academy.silver.products
UNION ALL
SELECT 
    'academy.silver.orders' AS table_name,
    COUNT(*) AS row_count
FROM academy.silver.orders
UNION ALL
SELECT 
    'academy.gold.dim_product' AS table_name,
    COUNT(*) AS row_count
FROM academy.gold.dim_product
UNION ALL
SELECT 
    'academy.gold.fact_sales' AS table_name,
    COUNT(*) AS row_count
FROM academy.gold.fact_sales;


## 📚 Additional Resources

- **Dashboard Layout Reference**: See `dashboard_layout_reference.md`
- **Dashboard Demo Guide**: See `dashboard_demo_guide.md`
- **Week 8 Assignment**: See `week8_real_world_assignment.md`

---

## 🎓 What You Learned

✅ Creating Gold-layer dimension and fact tables  
✅ Writing business analytics queries  
✅ Preparing data for dashboard visualizations  
✅ Building interactive Databricks dashboards  

**Next Steps**: Complete the Week 8 assignment to build your own dashboard!

---

*Clogenai Academy – Data Pipeline Engineering Track*  
*Week 8 Complete Dashboard Demo*  
*Version 1.0 – December 2025*
